In [1]:
#Use our Recommendation system to return the best recommend accommodation id

#import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#import accommodations data
items=pd.read_csv('/Users/loki/Downloads/9900/loki-recommedationsystem/accommodation.csv',sep=',')
items.head()

,id,host_id,name,num_guests,num_bedrooms,num_beds,num_bathrooms,description,suburb,city,state,country,price,property_type,amenities,image_urls,rating,num_reviews
0,101,1,An Oasis in the City,1,1,1,1.0,Very central to the city which can be reached ...,Potts Point,Potts Point,NSW,Australia,6400,Apartment,"[u'tv', u'kitchen', u'elevator', u'buzzer/wire...",[u'https://a0.muscache.com/im/pictures/2797669...,92,185
1,102,1,Sydney City & Harbour at the door,2,1,1,1.0,Come stay with Vinh & Stuart (Awarded as one o...,Pyrmont,Pyrmont,NSW,Australia,9900,Townhouse,"[u'tv', u'internet', u'wifi', u'air conditioni...",[u'https://a0.muscache.com/im/pictures/763ad5c...,95,510
2,103,1,Manly Harbour House,6,3,3,3.0,"Beautifully renovated, spacious and quiet, our...",Balgowlah,Balgowlah,NSW,Australia,47000,House,"[u'tv', u'wifi', u'air conditioning', u'kitche...",[u'https://a0.muscache.com/im/pictures/5693567...,90,2
3,104,1,Stunning Penthouse Apartment In Heart Of The City,1,1,1,1.0,Penthouse living in a great central location: ...,Darlinghurst,Darlinghurst,NSW,Australia,11000,Apartment,"[u'tv', u'cable tv', u'internet', u'wifi', u'a...",[u'https://a0.muscache.com/im/pictures/3643414...,88,321
4,105,1,3 BED HOUSE + 1 BED STUDIO Balmain,6,4,4,2.0,Hi! We are a married professional couple with ...,Balmain/Birchgrove,Balmain,NSW,Australia,45000,House,"[u'tv', u'internet', u'wifi', u'air conditioni...",[u'https://a0.muscache.com/im/pictures/5878512...,96,16


In [2]:
#show features of accommodations
col = list(items.columns)
print(col)
items_num = items[['id', 'num_guests', 'num_bedrooms', 'num_beds', 'num_bathrooms', 'price', 'rating', 'num_reviews']]

['id', 'host_id', 'name', 'num_guests', 'num_bedrooms', 'num_beds', 'num_bathrooms', 'description', 'suburb', 'city', 'state', 'country', 'price', 'property_type', 'amenities', 'image_urls', 'rating', 'num_reviews']


In [3]:
#import users data
users=pd.read_csv('/Users/loki/Downloads/9900/loki-recommedationsystem/user.csv',sep=',')
users.head()

,user_id,Gender,Age,occupation_id,zip_code
0,1,F,10,10,2017
1,2,M,56,1,2018
2,3,M,25,3,2003
3,4,M,45,1,2102
4,5,M,25,4,2223


In [4]:
movie_ratings = items[['id','rating']]
print(movie_ratings.sort_values(by=['rating'], ascending=False))

    id  rating
7  108      98
6  107      97
4  105      96
9  115      96
1  102      95
5  106      95
8  109      95
0  101      92
2  103      90
3  104      88


In [5]:
#set rating threshold
RatingsCount_threshold = 95
movie_ratings_popular=movie_ratings.query('rating>=@RatingsCount_threshold')
movie_ratings_popular

,id,rating
1,102,95
4,105,96
5,106,95
6,107,97
7,108,98
8,109,95
9,115,96


In [6]:
# Calculate C
C = items['rating'].mean()
C

94.2

In [7]:
items_num

,id,num_guests,num_bedrooms,num_beds,num_bathrooms,price,rating,num_reviews
0,101,1,1,1,1.0,6400,92,185
1,102,2,1,1,1.0,9900,95,510
2,103,6,3,3,3.0,47000,90,2
3,104,1,1,1,1.0,11000,88,321
4,105,6,4,4,2.0,45000,96,16
5,106,1,1,1,1.0,6100,95,45
6,107,2,0,1,1.0,15900,97,63
7,108,1,1,1,1.0,13000,98,60
8,109,2,1,1,1.0,17400,95,128
9,115,1,1,3,1.5,5400,96,165


In [8]:
#items_num.set_index(['id'])

items_num_norm = items_num[['num_guests', 'num_bedrooms', 'num_beds', 'num_bathrooms', 'price', 'rating', 'num_reviews']]
#items_num_norm.set_index(items_num['id'])
items_num_norm

,num_guests,num_bedrooms,num_beds,num_bathrooms,price,rating,num_reviews
0,1,1,1,1.0,6400,92,185
1,2,1,1,1.0,9900,95,510
2,6,3,3,3.0,47000,90,2
3,1,1,1,1.0,11000,88,321
4,6,4,4,2.0,45000,96,16
5,1,1,1,1.0,6100,95,45
6,2,0,1,1.0,15900,97,63
7,1,1,1,1.0,13000,98,60
8,2,1,1,1.0,17400,95,128
9,1,1,3,1.5,5400,96,165


In [9]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
standard_scaler = preprocessing.StandardScaler()
standard_matrix = standard_scaler.fit_transform(items_num)
standard_matrix

#Zscore-Normalization
#items_num_norm = (items_num - items_num.mean()) / (items_num.std())
#print(items_num_norm)

array([[-1.29099445, -0.68421053, -0.3592106 , -0.63636364, -0.55167728,
        -0.77161038, -0.72690046,  0.23545251],
       [-1.03279556, -0.15789474, -0.3592106 , -0.63636364, -0.55167728,
        -0.53282732,  0.26432744,  2.39100363],
       [-0.77459667,  1.94736842,  1.43684242,  1.18181818,  2.60076434,
         1.99827303, -1.38771906, -0.97828859],
       [-0.51639778, -0.68421053, -0.3592106 , -0.63636364, -0.55167728,
        -0.45778122, -2.04853766,  1.13746775],
       [-0.25819889,  1.94736842,  2.33486893,  2.09090909,  1.02454353,
         1.86182557,  0.59473674, -0.88543408],
       [ 0.        , -0.68421053, -0.3592106 , -0.63636364, -0.55167728,
        -0.79207749,  0.26432744, -0.69309259],
       [ 0.25819889, -0.15789474, -1.25723711, -0.63636364, -0.55167728,
        -0.12348495,  0.92514604, -0.57370822],
       [ 0.51639778, -0.68421053, -0.3592106 , -0.63636364, -0.55167728,
        -0.32133376,  1.25555534, -0.59360562],
       [ 0.77459667, -0.15789474

In [10]:
print(items_num.index)
print(items_num.columns)

RangeIndex(start=0, stop=10, step=1)
Index(['id', 'num_guests', 'num_bedrooms', 'num_beds', 'num_bathrooms',
       'price', 'rating', 'num_reviews'],
      dtype='object')


In [11]:
# Import cosine_score 
from sklearn.metrics.pairwise import cosine_similarity


#Compute the cosine similarity matrix using the dummy ratings matrix
cosine_sim = cosine_similarity(standard_matrix)

cosine_sim

array([[ 1.        ,  0.53118947, -0.40212102,  0.74302545, -0.64850949,
         0.48645096,  0.05790952, -0.04255991, -0.10354602, -0.51202908],
       [ 0.53118947,  1.        , -0.45610757,  0.51150314, -0.46046506,
        -0.04910311, -0.02656674, -0.09369122, -0.05574243, -0.26586274],
       [-0.40212102, -0.45610757,  1.        , -0.22555884,  0.79681454,
        -0.72628408, -0.62343725, -0.70877547, -0.67138056, -0.3134138 ],
       [ 0.74302545,  0.51150314, -0.22555884,  1.        , -0.5912344 ,
         0.07980767, -0.2655998 , -0.40236201, -0.04391436, -0.27977462],
       [-0.64850949, -0.46046506,  0.79681454, -0.5912344 ,  1.        ,
        -0.68835653, -0.5259584 , -0.42233411, -0.54691062, -0.10586423],
       [ 0.48645096, -0.04910311, -0.72628408,  0.07980767, -0.68835653,
         1.        ,  0.6734845 ,  0.7699986 ,  0.57607579,  0.1012626 ],
       [ 0.05790952, -0.02656674, -0.62343725, -0.2655998 , -0.5259584 ,
         0.6734845 ,  1.        ,  0.81928465

In [30]:
#Convert into pandas dataframe 
ac_id = np.array(items_num['id']).tolist()
ac_id

cosine_sim = pd.DataFrame(cosine_sim, index=ac_id, columns=ac_id)

cosine_sim


,101,102,103,104,105,106,107,108,109,115
101,1.000000,0.531189,-0.402121,0.743025,-0.648509,0.486451,0.057910,-0.042560,-0.103546,-0.512029
102,0.531189,1.000000,-0.456108,0.511503,-0.460465,-0.049103,-0.026567,-0.093691,-0.055742,-0.265863
103,-0.402121,-0.456108,1.000000,-0.225559,0.796815,-0.726284,-0.623437,-0.708775,-0.671381,-0.313414
104,0.743025,0.511503,-0.225559,1.000000,-0.591234,0.079808,-0.265600,-0.402362,-0.043914,-0.279775
105,-0.648509,-0.460465,0.796815,-0.591234,1.000000,-0.688357,-0.525958,-0.422334,-0.546911,-0.105864
106,0.486451,-0.049103,-0.726284,0.079808,-0.688357,1.000000,0.673484,0.769999,0.576076,0.101263
107,0.057910,-0.026567,-0.623437,-0.265600,-0.525958,0.673484,1.000000,0.819285,0.726858,0.157872
108,-0.042560,-0.093691,-0.708775,-0.402362,-0.422334,0.769999,0.819285,1.000000,0.747686,0.337466
109,-0.103546,-0.055742,-0.671381,-0.043914,-0.546911,0.576076,0.726858,0.747686,1.000000,0.361381
115,-0.512029,-0.265863,-0.313414,-0.279775,-0.105864,0.101263,0.157872,0.337466,0.361381,1.000000


In [36]:
#Item Based Similarity recommendation system
def item_based_recommendation(accommodation_id):
    similarity_list = cosine_sim[accommodation_id][1:].sort_values(ascending=False)
    return similarity_list

104    0.743025
102    0.531189
106    0.486451
107    0.057910
Name: 101, dtype: float64

In [37]:
item_based_recommendation(101)[0:4]

104    0.743025
102    0.531189
106    0.486451
107    0.057910
Name: 101, dtype: float64

In [42]:
recommend_accommodation_id = item_based_recommendation(101)[0:4]
list_of_recommend_accommodation_id = recommend_accommodation_id.index.tolist()
print(list_of_recommend_accommodation_id)

[104, 102, 106, 107]
